## Terms

### explantory variables
independent variables
### Collinearity
a linear association between two explantory variables: $X_{2i} = \lambda_0 + \lambda_1 X_{1i} $

### Multicollinearity
two or more explantory variables are highly linearly related: $\lambda_0 + \lambda_1 X_{1i} + \lambda_2 X_{2i} + \dots + \lambda_k X_{ki} = 0$


### degree of freedom
The minimum number of independent coordinates that can specify the position of the system completely.

See examples in the "Of random vectors" section from [Degrees of freedom(statistics)](https://en.wikipedia.org/wiki/Degrees_of_freedom_(statistics))

### singular matrix
A square matrix that does not have a matrix inverse

